In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/istywhyerlina/PipelinePyspark/exercise4/data_pipeline_exercise_4/script')
from helper.db_conn import db_connection
from pangres import upsert
from etl.extract import *
from etl.load import *
from etl.transform import *
from ML.split import *

from sqlalchemy import *



In [2]:
_, stg_engine, dwh_engine,_ = db_connection()
meta = MetaData()
meta.reflect(bind=stg_engine)
for table in meta.sorted_tables:
    globals()[f'{table}']=extract_db(str(table), "dwh")

Extract Data car_brand from Datawarehouse is Succeed
Extract Data car_sales from Datawarehouse is Succeed
Extract Data us_state from Datawarehouse is Succeed


In [3]:
car_brand=car_brand[['brand_car_id','brand_name']]
us_state=us_state[['id_state','name']]


In [4]:
car_sales =car_sales .merge(car_brand, how="left", on='brand_car_id')
car_sales = car_sales.merge(us_state, how="left", left_on='id_state', right_on='id_state')

In [5]:
car_sales=car_sales.drop(['brand_car_id','id_state','id_sales','id_sales_nk','created_at'], axis=1)
car_sales = car_sales.rename({'name': 'state'}, axis='columns')

In [7]:
X_train, X_test, y_train, y_test = splitting_process(data = car_sales,
                                                         target_col = 'selling_price',
                                                         test_size = 80)

25535    22000.0
17963    11100.0
25246     9000.0
5081      9901.0
9350      2800.0
          ...   
29802    26000.0
5390     14500.0
860      38400.0
15795     1900.0
23654    11600.0
Name: selling_price, Length: 29920, dtype: float64

In [24]:

# Perform one-hot encoding
car_sales2 = pd.get_dummies(car_sales, columns=['transmission','color','interior','brand_name','state'])

print(car_sales2)

       year  condition  odometer      mmr  selling_price  transmission_  \
0      2005       19.0   99483.0   3975.0         3500.0           True   
1      2012       43.0   30032.0  14800.0        14600.0          False   
2      2012       47.0   28408.0  25500.0        24500.0           True   
3      2014       46.0   13069.0  27000.0        26600.0          False   
4      2004       19.0  155121.0   1325.0          500.0           True   
...     ...        ...       ...      ...            ...            ...   
29995  2011       32.0  104005.0  10400.0         9500.0          False   
29996  2005        4.0   57868.0   7825.0        12200.0          False   
29997  2012       25.0   34425.0  27600.0        23900.0          False   
29998  2012       32.0   33334.0  17450.0        16800.0          False   
29999  2005        NaN  108430.0   3200.0         3100.0          False   

       transmission_Sedan  transmission_automatic  transmission_manual  \
0                   False